In [1]:
#creating spark session and importing libraries
import findspark
findspark.init("/opt/spark")  
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as fn
import os
from dotenv import load_dotenv


spark = SparkSession.builder \
    .appName("MySQL Integration") \
    .config("spark.jars", "mysql-connector-j-8.0.33.jar") \
    .config("spark.driver.extraClassPath", "mysql-connector-j-8.0.33.jar") \
    .config("spark.executor.extraClassPath", "mysql-connector-j-8.0.33.jar") \
    .getOrCreate()



25/03/16 19:45:02 WARN Utils: Your hostname, abanoub-Latitude-E5440 resolves to a loopback address: 127.0.1.1; using 192.168.1.4 instead (on interface wlp2s0)
25/03/16 19:45:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/16 19:45:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
#reading all files from the archive 
df_total=spark.read.json('archive/*.json')

In [3]:
#testing the reading 
df_total.show()

+-------------------+--------------------+-----+--------------------+---------+----------------+-----------+--------------------+-----+-----------+--------------------+------------+-------------------+
|            created|         description|forks|           full_name|       id|        language|open_issues|           repo_name|stars|subscribers|              topics|        type|           username|
+-------------------+--------------------+-----+--------------------+---------+----------------+-----------+--------------------+-----+-----------+--------------------+------------+-------------------+
|2014-02-25 08:00:08|Apache Spark - A ...|25357|        apache/spark| 17165658|           Scala|        242|               spark|32296|       2080|[python, scala, r...|Organization|             apache|
|2017-08-09 19:39:59|Distributed train...| 2027|     horovod/horovod| 99846383|          Python|        298|             horovod|12219|        334|[tensorflow, uber...|Organization|           

In [4]:
df_total.select('language').distinct().show(136)

+-----------------+
|         language|
+-----------------+
|               C#|
|            Terra|
|             Hack|
|         Makefile|
|              VBA|
|             Cuda|
|         Markdown|
|              Arc|
|             LLVM|
|            Metal|
|             Haxe|
|             Rust|
|       JavaScript|
|       Emacs Lisp|
|             TSQL|
|             Perl|
|           Puppet|
|           Erlang|
|          Crystal|
|            Jinja|
|              Nim|
|              C++|
|               F#|
|           Groovy|
|              TeX|
|            OCaml|
|             Dart|
|             MQL5|
|            Cirru|
|              Elm|
|              Vue|
|              Coq|
|       Dockerfile|
|     ActionScript|
|            Julia|
| Jupyter Notebook|
|             XSLT|
|       Vim script|
|    Objective-C++|
|                C|
|      Objective-C|
|          Verilog|
|       ApacheConf|
|            Frege|
|            Swift|
|       TypeScript|
|             NASL|


In [5]:
#creating view for first requistion (programming_lang) 
df_total.createOrReplaceTempView('lang')
programming_lang=spark.sql('Select language ,count(*) as number_of_repos From lang GROUP BY language')


In [6]:
#test 1st df 
programming_lang.show()
programming_lang.printSchema()

+----------+---------------+
|  language|number_of_repos|
+----------+---------------+
|        C#|            336|
|     Terra|              1|
|      Hack|              1|
|  Makefile|             36|
|       VBA|              3|
|      Cuda|             12|
|  Markdown|              3|
|       Arc|              1|
|      LLVM|              2|
|     Metal|              1|
|      Haxe|              4|
|      Rust|            165|
|JavaScript|           5293|
|Emacs Lisp|             18|
|      TSQL|              9|
|      Perl|             14|
|    Puppet|              4|
|    Erlang|              6|
|      NULL|           1424|
|   Crystal|              2|
+----------+---------------+
only showing top 20 rows

root
 |-- language: string (nullable = true)
 |-- number_of_repos: long (nullable = false)



In [7]:
#test2 for 1st df 
org=spark.sql('Select username ,stars From lang WHERE username=="facebookincubator"').show(n=10000)

25/03/16 19:45:17 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+-----------------+-----+
|         username|stars|
+-----------------+-----+
|facebookincubator|  375|
|facebookincubator| 9305|
|facebookincubator|  998|
|facebookincubator| 9304|
+-----------------+-----+



In [8]:
# getting the password for database from .env file
load_dotenv()
db_password = os.getenv("mysql_pass")

In [9]:
# writing 1st df into mysql db 
programming_lang.write \
        .format("jdbc") \
        .option("url", "jdbc:mysql://localhost:3306/Github_repos") \
        .option("dbtable", "programming_lang_database") \
        .option("user", "root") \
        .option("password", db_password) \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .mode("append") \
        .save()


In [10]:
#creating 2nd df 
organizations_stars=spark.sql('Select username ,sum(stars) as total_number_of_stars From lang GROUP BY username')

In [11]:
#test for 2nd df 
organizations_stars.show()

+----------------+---------------------+
|        username|total_number_of_stars|
+----------------+---------------------+
|       tribbloid|                  239|
|             P7h|                  386|
|       zaratsian|                   68|
|       jtnystrom|                    7|
|            didi|                 4914|
|          zdkzdk|                   49|
|       zhangslob|                   43|
|        deepmipt|                16875|
|        blei-lab|                14079|
|          GPflow|                 4786|
|        asingh33|                  843|
|        lambdaji|                  653|
|        TheLordA|                   53|
|         esoxjem|                 2571|
|    ITDragonBlog|                 1163|
|          anastr|                 1110|
|       wewewe718|                  323|
|        catchorg|                14637|
|AtomicGameEngine|                 5249|
|         p-ranav|                 5906|
+----------------+---------------------+
only showing top

In [12]:
print(organizations_stars)

DataFrame[username: string, total_number_of_stars: bigint]


In [13]:
#writing 2nd requistion into mysql db 
organizations_stars.write \
        .format("jdbc") \
        .option("url", "jdbc:mysql://localhost:3306/Github_repos") \
        .option("dbtable", "organizations_stars_database") \
        .option("user", "root") \
        .option("password",db_password) \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .mode("append") \
        .save()


In [14]:
#getting all directories for the json files 
import os 
main_directory='archive'
files=os.listdir(main_directory)
list_of_directories=[]
for file in files:
    directory_of_file=os.path.join(main_directory, file)
    list_of_directories.append(directory_of_file)

print(list_of_directories)

['archive/Angular.json', 'archive/Django.json', 'archive/Docker.json', 'archive/Gatsby.json', 'archive/React-Native.json', 'archive/Tensorflow.json', 'archive/Cpp.json', 'archive/Scala.json', 'archive/Machine-Learning.json', 'archive/Kubernetes.json', 'archive/ethereum.json', 'archive/PyTorch.json', 'archive/serverless.json', 'archive/Scikit.json', 'archive/NextJS.json', 'archive/Deep-Learning.json', 'archive/Kotlin.json', 'archive/React-JS.json', 'archive/Julia.json', 'archive/NodeJS.json', 'archive/Golang.json', 'archive/Threejs.json', 'archive/R.json', 'archive/Typescript.json', 'archive/Dart.json', 'archive/Flask.json', 'archive/Spark.json', 'archive/OOPs.json', 'archive/Hadoop.json']


In [15]:
# getting names for json files 
df_names = [os.path.splitext(file)[0] for file in files]
print(df_names)

['Angular', 'Django', 'Docker', 'Gatsby', 'React-Native', 'Tensorflow', 'Cpp', 'Scala', 'Machine-Learning', 'Kubernetes', 'ethereum', 'PyTorch', 'serverless', 'Scikit', 'NextJS', 'Deep-Learning', 'Kotlin', 'React-JS', 'Julia', 'NodeJS', 'Golang', 'Threejs', 'R', 'Typescript', 'Dart', 'Flask', 'Spark', 'OOPs', 'Hadoop']


In [16]:
#creation for dfs for every file
df_total_search=[]
for i, directory in enumerate(list_of_directories):
    search_term=df_names[i]
    df=spark.read.json(directory)
    df.createOrReplaceTempView('view_for_df')
    relevance_score= spark.sql(f'SELECT "{search_term}" as search_term, sum(forks)*1.5 + sum(subscribers)*1.32 + sum(stars)*1.04 as relevance_score FROM view_for_df WHERE type=="Organization"')
    df_total_search.append(relevance_score)

In [17]:
df_total_search[0].show()

+-----------+---------------+
|search_term|relevance_score|
+-----------+---------------+
|    Angular|     1777052.40|
+-----------+---------------+



In [18]:
#merging all dfs and names in one df 
merged_df = df_total_search[0]
for df in df_total_search[1:]: 
    merged_df = merged_df.union(df)

merged_df.show(30)

+----------------+---------------+
|     search_term|relevance_score|
+----------------+---------------+
|         Angular|     1777052.40|
|          Django|      898331.96|
|          Docker|     2780901.88|
|          Gatsby|      199343.26|
|    React-Native|     1686382.14|
|      Tensorflow|     1933000.62|
|             Cpp|     2037344.12|
|           Scala|      879416.38|
|Machine-Learning|     3663187.06|
|      Kubernetes|     2877257.56|
|        ethereum|      898290.74|
|         PyTorch|     1527742.68|
|      serverless|     1011225.28|
|          Scikit|      409319.20|
|          NextJS|      399003.86|
|   Deep-Learning|     3225845.64|
|          Kotlin|      995196.54|
|        React-JS|      412299.06|
|           Julia|      281220.12|
|          NodeJS|     2700362.96|
|          Golang|     3074290.76|
|         Threejs|       68310.94|
|               R|     6804028.24|
|      Typescript|     3741619.90|
|            Dart|      548056.92|
|           Flask|  

In [19]:
#writing all dfs 3rd requistion into mysql db 
merged_df.write \
        .format("jdbc") \
        .option("url", "jdbc:mysql://localhost:3306/Github_repos") \
        .option("dbtable", "Search_terms_relevance_database") \
        .option("user", "root") \
        .option("password", db_password) \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .mode("append") \
        .save()
